In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
import locale


def getpreferredencoding(do_setlocale=True):
    return "UTF-8"


locale.getpreferredencoding = getpreferredencoding
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 37.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount("/content/drive/")
gdrive_path = ""

Mounted at /content/drive/


In [ ]:
!ls {gdrive_path}

wiki_queries_101_646.parquet  wiki_queries_47_646.parquet
wiki_queries_10_646.parquet   wiki_queries_48_646.parquet
wiki_queries_11_646.parquet   wiki_queries_49_646.parquet
wiki_queries_121_646.parquet  wiki_queries_50_646.parquet
wiki_queries_12_646.parquet   wiki_queries_51_646.parquet
wiki_queries_13_646.parquet   wiki_queries_52_646.parquet
wiki_queries_141_646.parquet  wiki_queries_53_646.parquet
wiki_queries_14_646.parquet   wiki_queries_54_646.parquet
wiki_queries_15_646.parquet   wiki_queries_55_646.parquet
wiki_queries_1_646.parquet    wiki_queries_5_646.parquet
wiki_queries_16_646.parquet   wiki_queries_56_646.parquet
wiki_queries_17_646.parquet   wiki_queries_57_646.parquet
wiki_queries_18_646.parquet   wiki_queries_58_646.parquet
wiki_queries_19_646.parquet   wiki_queries_59_646.parquet
wiki_queries_22_646.parquet   wiki_queries_60_646.parquet
wiki_queries_23_646.parquet   wiki_queries_61_646.parquet
wiki_queries_24_646.parquet   wiki_queries_62_646.parquet
wiki_queries_25

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-22475aeb-c069-1641-b865-7bf148d2f49b)


In [ ]:
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained("castorini/doc2query-t5-base-msmarco", trust_remote_code=True)
model = T5ForConditionalGeneration.from_pretrained("castorini/doc2query-t5-base-msmarco", trust_remote_code=True)
model.to(device)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
def doc_2_query(doc_text):
    input_ids = tokenizer.encode(doc_text, return_tensors="pt").to(device)
    outputs = model.generate(input_ids=input_ids, max_length=64, do_sample=True, top_k=10, num_return_sequences=3)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [ ]:
wiki_df = pd.read_parquet(gdrive_path + "wiki_trimmed.parquet")
wiki_df

,id,url,text,title,word_num
0,12,https://en.wikipedia.org/wiki/Anarchism,Anarchism is a political philosophy and moveme...,Anarchism,313
1,25,https://en.wikipedia.org/wiki/Autism,Autism is a neurodevelopmental disorder charac...,Autism,349
2,39,https://en.wikipedia.org/wiki/Albedo,Albedo is the measure of the diffuse reflecti...,Albedo,318
3,290,https://en.wikipedia.org/wiki/A,"A, or a, is the first letter and the first vow...",A,121
4,303,https://en.wikipedia.org/wiki/Alabama,Alabama is a state in the Southeastern region...,Alabama,527
...,...,...,...,...,...
6458665,70201819,https://en.wikipedia.org/wiki/Bianca%20Fernandez,Bianca Jolie Fernandez is a Canadian tennis p...,Bianca Fernandez,85
6458666,70201882,https://en.wikipedia.org/wiki/Condons%20and%20...,Condons and Clangibbon is a barony in County ...,Condons and Clangibbon,13
6458667,70201886,https://en.wikipedia.org/wiki/2022%20Chattanoo...,The 2022 Chattanooga Red Wolves SC season will...,2022 Chattanooga Red Wolves SC season,85
6458668,70201947,https://en.wikipedia.org/wiki/Nkiko%20Prosper,Turatsinze Nkiko Prosper professionally known...,Nkiko Prosper,72


In [ ]:
doc_2_query(wiki_df.iloc[4]["text"])

Token indices sequence length is longer than the specified maximum sequence length for this model (728 > 512). Running this sequence through the model will result in indexing errors


['what year was the alabama state recognized as a state',
 'when was alabama recognized as a state?',
 'why is alabama a major state']

In [ ]:
batch_size = 10000
# wiki_test_df = wiki_df.head(5)
# batch_df_list = []
len_batches = len(np.unique(np.arange(len(wiki_df)) // batch_size))
for batch_number, batch_df in wiki_df.groupby(np.arange(len(wiki_df)) // batch_size):
    if batch_number + 1 <= 79:
        continue
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Processing {batch_number+1}/{len_batches}")
    batch_query_list = []
    for index, row in batch_df.iterrows():
        # if index % 100 == 0:
        # print(f"   [{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] i={index}")
        queries = doc_2_query(row["text"])
        batch_query_list.append(queries)
    batch_df = batch_df.assign(queries=batch_query_list)
    batch_df.to_parquet(gdrive_path + f"wiki_queries_{batch_number+1}_{len_batches}.parquet")
    # batch_df_list.append(batch_df)
# batch_df_list[0]

[2023-03-20 13:23:45] Processing 80/646
[2023-03-20 14:04:44] Processing 81/646
[2023-03-20 14:45:03] Processing 82/646
[2023-03-20 15:24:52] Processing 83/646
[2023-03-20 16:04:32] Processing 84/646
[2023-03-20 16:44:26] Processing 85/646
